In [1]:
# make simulated data. The assumptions are thaat the data is generated as follows:
# I have written the below as equalities, but in reality they are random variables.

# N_i = Poisson(lambda_i) . the Number of birds at site i is a Poisson random variable with mean lambda_i
# p_ij = Beta(alpha, beta) . The singing rate (p of singing in a given 5s window) of a bird at site i on day j is a Beta random variable with parameters alpha and beta
# n_song_clips_ij ~ Binomial(N_i, p_ij) . The number of song clips recorded at site i on day j is a Binomial random variable with parameters N_i and p_ij


In [26]:
def make_simulated_data(pop_lambda, p_call, nsites, nclips, pos_mu, pos_sd, neg_mu, neg_sd):
    """
    Make simulated data. The assumptions are thaat the data is generated as follows:
    Number of birds at site i is a Poisson random variable with mean pop_lambda
    The singing rate (p of singing in a given 5s window) is p_call
    The number of song clips recorded at site i on day j is a Binomial random variable with parameters N_i and p_ij
    The positive and negative gaussian parameters are pos_gauss_params (mu_1, sd_1) and neg_gauss_params (mu2, sd2)
    """
    import numpy as np
    import pandas as pd
    import scipy.stats as stats
    from scipy.stats import poisson, beta, binom
    import random

    # make the population sizes for each site
    N_birds = poisson.rvs(pop_lambda, size=nsites)

    p_nocall = (1 - p_call)**N_birds # p of a clip containing no calls

    # number of clips containing no calls at each site
    num_nocall_clips = np.random.binomial(nclips, p_nocall)

    # now make an array where each row is a site, and each column is a clip
    # each entry is the score for that clip
    # the score is a random variable from a mixture of two gaussians
    # the first gaussian is the positive gaussian, the second is the negative gaussian
    # so draw num_nocall_clips from the negative gaussian, and nclips - num_nocall_clips from the positive gaussian
    scores = np.zeros((nsites, nclips))
    for i in range(nsites):
        scores[i, :num_nocall_clips[i]] = np.random.normal(neg_mu, neg_sd, num_nocall_clips[i])
        scores[i, num_nocall_clips[i]:] = np.random.normal(pos_mu, pos_sd, nclips - num_nocall_clips[i])

    # make the data frame
    scores_df = pd.DataFrame(scores)

    # now make the truth_df, for storing the true values of N, p_call, and num_nocall_clips, pos_mu, pos_sd, neg_mu, neg_sd
    truth_df = pd.DataFrame({'N_birds': N_birds, 'p_call': p_call, 'num_nocall_clips': num_nocall_clips, 'pos_mu': pos_mu, 'pos_sd': pos_sd, 'neg_mu': neg_mu, 'neg_sd': neg_sd})

    return scores_df, truth_df

In [35]:
NSITES = 100
NCLIPS = 720
pos_mu = 5
pos_sd = 2
neg_mu = -1
neg_sd = 1
scores_df, truth_df = make_simulated_data(pop_lambda=2, p_call = 0.1, 
                                          nsites = NSITES, nclips = NCLIPS, pos_mu = pos_mu, 
                                          pos_sd = pos_sd, neg_mu = neg_mu, neg_sd = neg_sd)

In [37]:
import numpy as np
for pop_lambda in np.arange(0.05, 3, 0.1):
    for p_call in np.arange(0.01, 0.5, 0.05):
        scores_df, truth_df = make_simulated_data(pop_lambda=pop_lambda, p_call = p_call, nsites = NSITES, nclips = NCLIPS, 
                                                  pos_mu = pos_mu, pos_sd = pos_sd, neg_mu = neg_mu, neg_sd = neg_sd)
        scores_df.to_csv(f'simulated_data/simulated_data_lambda_{pop_lambda:.2f}_p_call_{p_call:.2f}.csv')
        truth_df.to_csv(f'simulated_data/simulated_truthlambda_{pop_lambda:.2f}_p_call_{p_call:.2f}.csv')